# Utilisation de Bokeh pour des données géographiques

On utilise le même dataset airbnb de New York que pour geopandas.

## Afficher des points sur une carte

On charge le dataset :

In [1]:
import pandas as pd
data = pd.read_csv("AB_NYC_2019.csv")

On crée notre première figure bokeh.

In [2]:
from bokeh.plotting import figure, save
p = figure(title="AirBNB in New York")

Ici, on n'utilise pas un point comme dans geopandas, mais directement les coordonnées x et y de chacun des points.

In [3]:
p.circle(x=data['latitude'], y=data['longitude'], size=5, color="red")

GlyphRenderer(id='1038', ...)

Afin d'afficher un graph bokeh dans un notebook, la commande suivante est nécessaire :

In [4]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

On peut maintenant afficher notre nuage de points :

In [5]:
from bokeh.io import show
show(p)

Comme avec geopandas, le nuage de points sans une carte en fond ne nous donne pas beaucoup d'informations. Bokeh permet de charger automatiquement des cartes OpenStreetMap

In [6]:
from bokeh.tile_providers import get_provider, Vendors

In [7]:
p = figure(x_range=(-8250000, -8220000), y_range=(4960000, 4980000),
           x_axis_type="mercator", y_axis_type="mercator", title="NYC Airbnb")

p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

TileRenderer(id='1142', ...)

In [8]:
show(p)

Nous avons une carte de New York sur laquelle nous pouvons afficher nos locations. Attention, dans nos données, les points sont définis en latitude/longitude, alors que notre carte utilise une projection de mercator. On va utiliser pyproj qui permet de convertir d'un type de projection à l'autre :

In [9]:
import pyproj

project_projection = pyproj.Proj("+init=EPSG:4326")  # wgs84
osm_projection = pyproj.Proj("+init=EPSG:3857")  # default openstreetmap projection

data['x'], data['y'] = pyproj.transform(project_projection, osm_projection, list(data['longitude']), list(data['latitude']))

Les colonnes x et y de notre dataset contiennent les coordonnées de nos locations, il ne reste plus qu'à les afficher.

In [10]:
p.circle(x=data['x'], y=data['y'], size=1,fill_alpha=0.05)

GlyphRenderer(id='1199', ...)

In [11]:
show(p)

On voit que notre nuage de point s'accorde très bien à la carte. L'intérêt de bokeh réside dans la visualisation interactive. On va faire un premier exemple en affichant des informations supplémentaires au survol de la souris.

In [12]:
data_filtered = data[data['host_id'] <= 600000] #on filtre pour éviter d'avoir trop de points
# On normalise les prix
price_min = data_filtered['price'].min()
price_max = data_filtered['price'].max()
data_filtered['normed_price'] = (data_filtered['price']-price_min)/(price_max-price_min) * 100 + 1
# On définit une couleur pour chaque type de location
data_filtered['color'] = data_filtered['room_type']\
                        .replace('Private room','red')\
                        .replace('Entire home/apt','green')\
                        .replace('Shared room','yellow')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Afin d'afficher les informations des locations, il faut transformer notre dataframe en ColumnDataSource que Bokeh sait utiliser. On crée également un hovertool (qui réagit au survol de la souris).

In [13]:
from bokeh.models import ColumnDataSource, HoverTool
source = ColumnDataSource(data_filtered)
hover = HoverTool(tooltips=[
    ("Nom :", "@name"),
    ("Prix : ", "@price"),
    ("Room type : ","@room_type")
])

On recrée notre carte quasiment de la même manière, mais cette fois, on indique que la source à utiliser pour les cercles est notre ColumnDataSource.

In [14]:
p = figure(x_axis_type="mercator", y_axis_type="mercator", title="NYC Airbnb",tools=[hover,'box_zoom','pan', 'wheel_zoom','save'])

p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

p.circle(x='x',
         y='y', 
         source=source,
         size='normed_price',
         color='color',
         fill_alpha=0.05)

GlyphRenderer(id='1314', ...)

In [15]:
show(p)

Le résultat est une carte interactive avec des couleurs différentes suivant le type de location et des cercles dont la taille est proportionnelle au prix de la location.